## Votre mission ##

La détection des spams est l'une des applications du Machine Learning. La quasi-totalité des principaux fournisseurs de services de messagerie ont des systèmes de détection de spam intégrés et classent automatiquement ces messages comme « Courrier indésirable ».

Dans cette mission, nous utiliserons l'algorithme Naive Bayes pour créer un modèle qui peut classer les messages SMS [dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) comme spam ou non spam, en fonction de l'apprentissage que nous donnons au modèle. Il est important d'avoir un certain niveau d'intuition quant à ce à quoi pourrait ressembler un message texte spam. Ils contiennent généralement des mots comme « gratuit », « gagnant », « argent comptant », « prix » et autres, car ces textes sont conçus pour attirer votre attention et, dans un certain sens, vous inciter à les ouvrir. De plus, les messages de spam ont tendance à avoir des mots écrits en majuscules et ont également tendance à utiliser beaucoup de points d'exclamation. Pour le destinataire, il est généralement assez simple d'identifier un texte de spam et notre objectif ici est de former un modèle pour le faire pour nous !

Être capable d'identifier les messages de spam est un problème de classification binaire car les messages sont classés comme « Spam » ou « Pas spam » et rien d'autre. En outre, il s'agit d'un problème d'apprentissage supervisé, car nous alimenterons le modèle avec un ensemble de données étiqueté, dont il pourra tirer des leçons pour faire des prédictions futures.

### Étape 0 : Introduction au théorème naïf de Bayes ###

Le théorème de Bayes est l'un des premiers algorithmes d'inférence probabilistes développés par le révérend Bayes et fonctionne toujours extrêmement bien pour certains cas d'utilisation.

Il est préférable de comprendre ce théorème à l'aide d'un exemple. Disons que vous êtes membre des services secrets et que vous avez été déployé pour protéger le candidat à la présidentielle lors d'un de ses discours de campagne. Étant un événement public ouvert à tous, votre travail n'est pas facile et vous devez être constamment à l'affût des menaces. Donc, un point de départ est de mettre un certain facteur de menace pour chaque personne. Donc, en fonction des caractéristiques d'un individu, comme l'âge, le sexe et d'autres facteurs plus petits comme la personne porte-t-elle un sac ?, la personne a-t-elle l'air nerveuse ? etc. vous pouvez juger si cette personne est une menace viable.

Si un individu coche toutes les cases jusqu'à un niveau où il franchit un seuil de doute dans votre esprit, vous pouvez prendre des mesures et retirer cette personne du voisinage. Le théorème de Bayes fonctionne de la même manière que nous calculons la probabilité d'un événement (une personne étant une menace) en fonction des probabilités de certains événements liés (âge, sexe, présence de sac ou non, nervosité etc. de la personne) .

Une chose à considérer est l'indépendance de ces caractéristiques les unes par rapport aux autres. Par exemple, si un enfant semble nerveux à l'événement, la probabilité que cette personne soit une menace n'est pas aussi grande que si c'était un homme adulte qui était nerveux. Pour décomposer cela un peu plus, voici deux caractéristiques que nous considérons, l'âge ET la nervosité. Supposons que nous examinions ces caractéristiques individuellement, nous pourrions concevoir un modèle qui signale TOUTES les personnes nerveuses comme des menaces potentielles. Cependant, il est probable que nous ayons beaucoup de faux positifs car il y a de fortes chances que les mineurs présents à l'événement soient nerveux. Par conséquent, en tenant compte de l'âge d'une personne ainsi que de la fonction « nervosité », nous obtiendrions certainement un résultat plus précis quant à savoir qui sont des menaces potentielles et qui ne le sont pas.

C'est la partie « naïve » du théorème où il considère que chaque caractéristique est indépendante les unes des autres, ce qui n'est pas toujours le cas et peut donc affecter le jugement final.

En bref, le théorème de Bayes calcule la probabilité qu'un certain événement se produise (dans notre cas, un message étant du spam) sur la base des distributions probabilistes conjointes de certains autres événements (dans notre cas, un message étant classé comme spam). 

### Étape 1.1 : Comprendre notre ensemble de données ###


Nous utiliserons un [Dataset](https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection) du référentiel UCI Machine Learning qui contient une très bonne collection d'ensembles de données à des fins de recherche expérimentale. Le lien de données direct est (https://archive.ics.uci.edu/ml/machine-learning-databases/00228/).


 ** Voici un aperçu des données : ** 

<img src="images/dqnb.png" height="1242" width="1242">

Les colonnes de l'ensemble de données ne sont actuellement pas nommées et comme vous pouvez le voir, il y a 2 colonnes.

La première colonne prend deux valeurs, 'ham' qui signifie que le message n'est pas du spam et 'spam' qui signifie que le message est un spam.

La deuxième colonne est le contenu textuel du message SMS en cours de classification.


* Importez l'ensemble de données dans un dataframe pandas à l'aide de la méthode read_table. Comme il s'agit d'un ensemble de données séparé par des tabulations, nous utiliserons '\t' comme valeur pour l'argument 'sep' qui spécifie ce format.
* Aussi, renommez les noms de colonnes en spécifiant une liste ['label, 'sms_message'] à l'argument 'names' de read_table().
* Imprimez les cinq premières valeurs du dataframe avec les nouveaux noms de colonnes.

In [2]:

import pandas as pd
# Dataset à partir - https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
Dataset = "./smsspamcollection/SMSSpamCollection"
df = pd.read_csv(Dataset, delimiter='\t', names=["type de msg", "message"])
# Imprimez les cinq premières valeurs du dataframe
print(df.head())


  type de msg                                            message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...


### Étape 1.2 : Prétraitement des données ###

Maintenant que nous avons une compréhension de base de ce à quoi ressemble notre ensemble de données, convertissons nos étiquettes en variables binaires, 0 pour représenter 'ham' et 1 pour représenter 'spam' pour faciliter le calcul.

Vous vous demandez peut-être pourquoi devons-nous effectuer cette étape ? La réponse à cela réside dans la façon dont scikit-learn gère les entrées. Scikit-learn ne traite que les valeurs numériques et donc si nous devions laisser nos valeurs d'étiquette sous forme de chaînes, scikit-learn ferait la conversion en interne (plus précisément, les étiquettes de chaîne seront converties en valeurs flottantes inconnues).

Notre modèle serait toujours capable de faire des prédictions si nous laissions nos étiquettes sous forme de chaînes, mais nous pourrions rencontrer des problèmes plus tard lors du calcul des mesures de performance, par exemple lors du calcul de nos scores de précision et de rappel. Par conséquent, pour éviter des « pièges » inattendus plus tard, il est recommandé d'introduire nos valeurs catégorielles dans notre modèle sous forme d'entiers.


* Convertissez les valeurs de la colonne « étiquette » en valeurs numériques en utilisant la méthode de mappage comme suit :
{'ham':0, 'spam':1} Ceci mappe la valeur 'ham' à 0 et la valeur 'spam' à 1.
* Aussi, pour avoir une idée de la taille de l'ensemble de données que nous traitons, imprimez le nombre de lignes et de colonnes.

In [3]:
 
df['label'] = df['type de msg'].map({'ham': 0, 'spam': 1})
print(df.shape)
df.head()

(5572, 3)


,type de msg,message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


### Étape 2.1: Bag of words ###

Ce que nous avons ici dans notre dataset est une collection de données textuelles (5 572 lignes de données). La plupart des algorithmes de ML s'appuient sur des données numériques à introduire en tant qu'entrée, et les messages e-mail/sms sont généralement riches en texte.

Ici, nous aimerions présenter le concept de sac de mots (BoW) qui est un terme utilisé pour spécifier les problèmes qui ont un « sac de mots » ou une collection de données textuelles avec lesquelles il faut travailler. L'idée de base de BoW est de prendre un morceau de texte et de compter la fréquence des mots dans ce texte. Il est important de noter que le concept BoW traite chaque mot individuellement et que l'ordre dans lequel les mots apparaissent n'a pas d'importance.

En utilisant un processus que nous allons suivre maintenant, nous pouvons convertir une collection de documents en une matrice, chaque document étant une ligne et chaque mot (jeton) étant la colonne, et les valeurs correspondantes (ligne, colonne) étant la fréquence de occurrence de chaque mot ou signe dans ce document.

Par exemple:

Disons que nous avons 4 documents comme suit :

`['Hello, how are you!',
'Win money, win from home.',
'Call me now',
'Hello, Call you tomorrow?']`

Notre objectif ici est de convertir cet ensemble de textes en une matrice de distribution de fréquence, comme suit :

<img src="images/countvectorizer.png" height="542" width="542">

Ici, comme nous pouvons le voir, les documents sont numérotés dans les lignes et chaque mot est un nom de colonne, la valeur correspondante étant la fréquence de ce mot dans le document.

Décomposons cela et voyons comment nous pouvons effectuer cette conversion en utilisant un petit ensemble de documents.

Pour gérer cela, nous utiliserons sklearns
[count vectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer) qui effectue les opérations suivantes :

* Il tokenise la chaîne (sépare la chaîne en mots individuels) et donne un ID entier à chaque jeton.
* Il compte l'occurrence de chacun de ces jetons.

** Veuillez noter: **

* La méthode CountVectorizer convertit automatiquement tous les mots tokenisés en minuscules afin qu'elle ne traite pas les mots comme`He` et `he` différemment. Il le fait en utilisant le paramètre `lowercase` qui est défini par défaut sur `True`.

* Il ignore également toutes les ponctuations afin que les mots suivis d'un signe de ponctuation (par exemple : 'hello!') ne soient pas traités différemment des mêmes mots non préfixés ou suffixés par un signe de ponctuation (par exemple : 'hello'). Il le fait en utilisant le paramètre `token_pattern` qui a une expression régulière par défaut qui sélectionne des jetons de 2 caractères alphanumériques ou plus.

* Le troisième paramètre à prendre en compte est le paramètre `stop_words`. Les mots vides font référence aux mots les plus couramment utilisés dans une langue. Ils incluent des mots comme 'am', 'an', 'and', 'the', etc. En définissant cette valeur de paramètre sur `english`, CountVectorizer ignorera automatiquement tous les mots (de notre texte d'entrée) qui se trouvent dans la liste de mots vides en anglais dans scikit-learn. Ceci est extrêmement utile car les mots vides peuvent fausser nos calculs lorsque nous essayons de trouver certains mots clés indiquant un spam.

Nous plongerons dans l'application de chacun de ces éléments dans notre modèle dans une étape ultérieure, mais pour l'instant, il est important d'être conscient des techniques de prétraitement dont nous disposons lorsque nous traitons des données textuelles.

### Étape 2.2 : Implémenter Bag of Words à partir de zéro ###

Avant de plonger dans la bibliothèque Bag of Words (BoW) de scikit-learn pour faire le sale boulot à notre place, implémentons-la nous-mêmes d'abord afin que nous puissions comprendre ce qui se passe dans les coulisses.

** Étape 1 : Convertissez toutes les chaînes en minuscules. **

Disons que nous avons un ensemble de documents :

```
documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']
```
>>** Instructions : *** 
Convertissez toutes les chaînes des documents en minuscules. Enregistrez-les dans une liste appelée 'lower_case_documents'. Vous pouvez convertir des chaînes en minuscules en python en utilisant la méthode lower().

In [4]:

documents = ['Hello, how are you!',
             'Win money, win from home.',
             'Call me now.',
             'Hello, Call hello you tomorrow?']

lower_case_documents = [doc.lower() for doc in documents]
#TODO
print(lower_case_documents)

['hello, how are you!', 'win money, win from home.', 'call me now.', 'hello, call hello you tomorrow?']


** Étape 2 : Suppression de toutes les ponctuations **

>>**Instructions : **
Supprimez toute ponctuation des chaînes de l'ensemble de documents. Enregistrez-les dans une liste appelée 'sans_punctuation_documents'.

In [5]:
sans_punctuation_documents = []
import string

#TODO
sans_punctuation_documents = [''.join([char for char in doc if char not in string.punctuation]) for doc in lower_case_documents ]
print(sans_punctuation_documents)

['hello how are you', 'win money win from home', 'call me now', 'hello call hello you tomorrow']


** Étape 3 : Tokenisation **

Tokeniser une phrase dans un ensemble de documents signifie diviser une phrase en mots individuels à l'aide d'un délimiteur. Le délimiteur spécifie quel caractère nous utiliserons pour identifier le début et la fin d'un mot (par exemple, nous pourrions utiliser un seul espace comme délimiteur pour identifier les mots dans notre ensemble de documents.)

>>**Instructions :**
Tokeniser les chaînes stockées dans 'sans_punctuation_documents' en utilisant la méthode split(). et stocker le jeu de documents final
dans une liste appelée 'preprocessed_documents'.

In [7]:
preprocessed_documents = []
#TODO
preprocessed_documents = [doc.split() for doc in sans_punctuation_documents]

print(preprocessed_documents)

[['hello', 'how', 'are', 'you'], ['win', 'money', 'win', 'from', 'home'], ['call', 'me', 'now'], ['hello', 'call', 'hello', 'you', 'tomorrow']]


** Étape 4 : Compter les fréquences **

Maintenant que nous avons notre ensemble de documents dans le format requis, nous pouvons procéder au comptage de l'occurrence de chaque mot dans chaque document de l'ensemble de documents. Nous utiliserons la méthode `Counter` de la bibliothèque Python `collections` à cette fin.

`Counter` compte l'occurrence de chaque élément de la liste et renvoie un dictionnaire avec la clé comme élément compté et la valeur correspondante étant le nombre de cet élément dans la liste.

>>**Instructions :**
En utilisant la méthode Counter() et preprocessed_documents comme entrée, créez un dictionnaire avec les clés correspondant à chaque mot de chaque document et les valeurs correspondantes à la fréquence d'occurrence de ce mot. Enregistrez chaque dictionnaire de compteurs en tant qu'élément dans une liste appelée " Frequency_list ".


In [8]:
frequency_list = []
import pprint
from collections import Counter

#TODO
frequency_list = [Counter(doc) for doc in preprocessed_documents]
pprint.pprint(frequency_list)

[Counter({'hello': 1, 'how': 1, 'are': 1, 'you': 1}),
 Counter({'win': 2, 'money': 1, 'from': 1, 'home': 1}),
 Counter({'call': 1, 'me': 1, 'now': 1}),
 Counter({'hello': 2, 'call': 1, 'you': 1, 'tomorrow': 1})]


### Étape 2.3 : Implémentation de Bag of Words dans scikit-learn ###

Maintenant que nous avons implémenté le concept BoW à partir de zéro, allons de l'avant et utilisons scikit-learn pour effectuer ce processus de manière claire et succincte. Nous utiliserons le même ensemble de documents que celui utilisé à l'étape précédente.

In [ ]:

documents = ['Hello, how are you!',
                'Win money, win from home.',
                'Call me now.',
                'Hello, Call hello you tomorrow?']

>>**Instructions :**
Importez la méthode sklearn.feature_extraction.text.CountVectorizer et créez-en une instance appelée 'count_vector'.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()



** Prétraitement des données avec CountVectorizer() **

À l'étape 2.2, nous avons implémenté une version de la méthode CountVectorizer() à partir de zéro qui impliquait d'abord de nettoyer nos données. Ce nettoyage impliquait de convertir toutes nos données en minuscules et de supprimer tous les signes de ponctuation. CountVectorizer() a certains paramètres qui prennent en charge ces étapes pour nous. Elles sont:

* `lowercase = True`
    
    Le paramètre `lowercase` a une valeur par défaut de « True » qui convertit tout notre texte en sa forme minuscule.


* `token_pattern = (?u)\\b\\w\\w+\\b`
    
    Le paramètre `token_pattern` a une valeur d'expression régulière par défaut de `(?u)\\b\\w\\w+\\b` qui ignore tous les signes de ponctuation et les traite comme des délimiteurs, tout en acceptant des chaînes alphanumériques de longueur supérieure ou égal à 2, sous forme de jetons ou de mots individuels.


* `stop_words`

    Le paramètre `stop_words`, s'il est défini sur `english`, supprimera tous les mots de notre ensemble de documents qui correspondent à une liste de mots vides anglais définie dans scikit-learn. Compte tenu de la taille de notre ensemble de données et du fait que nous traitons des messages SMS et non des sources de texte plus volumineuses comme les e-mails, nous ne définirons pas cette valeur de paramètre.

>>**Instructions :**
Ajustez votre ensemble de données de document à l'objet CountVectorizer que vous avez créé à l'aide de fit() et obtenez la liste des mots qui ont été classés en tant qu'entités à l'aide de la méthode get_feature_names().

In [12]:
count_vector.fit(documents)
feature_names = count_vector.get_feature_names_out()
print(feature_names)

['are' 'call' 'from' 'hello' 'home' 'how' 'me' 'money' 'now' 'tomorrow'
 'win' 'you']


>>**
Instructions:**
Créez une matrice avec les lignes étant chacun des 4 documents et les colonnes étant chaque mot.
La valeur correspondante (ligne, colonne) est la fréquence d'occurrence de ce mot (dans la colonne) dans un document particulier (dans la ligne). Vous pouvez le faire en utilisant la méthode transform() et en passant l'ensemble de données du document comme argument. La méthode transform() renvoie une matrice d'entiers numpy, vous pouvez la convertir en un tableau en utilisant toarray(). Appelez le tableau 'doc_array'


In [13]:
import numpy as np
doc_array = count_vector.transform(documents).toarray()
print(doc_array)

[[1 0 0 1 0 1 0 0 0 0 0 1]
 [0 0 1 0 1 0 0 1 0 0 2 0]
 [0 1 0 0 0 0 1 0 1 0 0 0]
 [0 1 0 2 0 0 0 0 0 1 0 1]]


Nous avons maintenant une représentation claire des documents en termes de distribution de fréquence des mots qu'ils contiennent. Pour faciliter la compréhension, notre prochaine étape consiste à convertir ce tableau en un dataframe et à nommer les colonnes de manière appropriée.

>>**Instructions :**
Convertissez le tableau que nous avons obtenu, chargé dans 'doc_array', en un dataframe et définissez les noms de colonne sur
les noms de mots (que vous avez eu précédemment à l'aide de get_feature_names(). Appelez le dataframe 'frequency_matrix'.


In [14]:
frequency_matrix = pd.DataFrame(doc_array, columns=count_vector.get_feature_names_out())
print(frequency_matrix)


   are  call  from  hello  home  how  me  money  now  tomorrow  win  you
0    1     0     0      1     0    1   0      0    0         0    0    1
1    0     0     1      0     1    0   0      1    0         0    2    0
2    0     1     0      0     0    0   1      0    1         0    0    0
3    0     1     0      2     0    0   0      0    0         1    0    1


Vous avez implémenté avec succès un problème de bag of words pour un ensemble de données de document que nous avons créé.

Un problème potentiel qui peut découler de l'utilisation de cette méthode prête à l'emploi est le fait que si notre ensemble de données de texte est extrêmement volumineux, certaines valeurs seront plus commune que d'autres simplement en raison de la structure de la langue elle-même. Ainsi, par exemple, des mots comme 'is', 'the', 'an', des pronoms, des constructions grammaticales, etc. pourraient fausser notre matrice et affecter notre analyse.

Il existe plusieurs façons d'atténuer cela. Une façon est d'utiliser le paramètre `stop_words` et de définir sa valeur sur `english`. Cela ignorera automatiquement tous les mots (de notre texte d'entrée) qui se trouvent dans une liste intégrée de mots vides en anglais dans scikit-learn.

Une autre façon d'atténuer cela consiste à utiliser la méthode [tfidf](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Cette méthode est hors de portée pour le contexte de cette manipulation.

### Étape 3.1 : Ensembles d'apprentissage et de test ###

Maintenant que nous avons compris comment traiter le problème du sac de mots, nous pouvons revenir à notre ensemble de données et poursuivre notre analyse. Notre première étape à cet égard serait de diviser notre ensemble de données en un ensemble d'entraînement et de test afin que nous puissions tester notre modèle plus tard.

>>**Instructions :**
Divisez l'ensemble de données en un ensemble d'apprentissage et de test à l'aide de la méthode train_test_split dans sklearn. Divisez les données à l'aide des variables suivantes :
-`X_train` est nos données d'entraînement pour la colonne 'sms_message'.
-`y_train` est nos données d'entraînement pour la colonne 'label'
-`X_test` est nos données de test pour la colonne 'sms_message'.
-`y_test` est nos données de test pour la colonne 'label'.
>>**
Imprimez le nombre de lignes que nous avons dans chacune de nos données d'entraînement et de test.


In [20]:
from sklearn.model_selection import train_test_split

# Division des données en ensemble d'entraînement et ensemble de test
X_train, X_test, y_train, y_test = train_test_split(df['sms_message'], 
                                                    df['label'], 
                                                    random_state=1)

print("Nombre de lignes dans l'ensemble total:", df.shape[0])
print("Nombre de lignes dans l'ensemble d'entraînement :", X_train.shape[0])
print("Nombre de lignes dans l'ensemble de test :", X_test.shape[0])


Nombre de lignes dans l'ensemble total: 5572
Nombre de lignes dans l'ensemble d'entraînement : 4179
Nombre de lignes dans l'ensemble de test : 1393


### Étape 3.2 : Application du traitement Bag of Words à notre ensemble de données. ###

Maintenant que nous avons divisé les données, notre prochain objectif est de suivre les étapes de l'étape 2 : Bag of words et de convertir nos données dans le format de matrice souhaité. Pour ce faire, nous utiliserons CountVectorizer() comme nous l'avons fait auparavant. Il y a deux étapes à considérer ici :

* Premièrement, nous devons ajuster nos données d'entraînement (`X_train`) dans `CountVectorizer()` et renvoyer la matrice.
* Deuxièmement, nous devons transformer nos données de test (`X_test`) pour renvoyer la matrice.

In [21]:
# Instancier la méthode CountVectorizer
# TODO
count_vector = CountVectorizer(lowercase=True)
# Ajuster les données d'entraînement, puis retourner la matrice
# TODO
X_train_C = count_vector.fit_transform(X_train)
# Transformez les données de test et renvoyez la matrice. Notez que nous n'ajustons pas les données de test dans le CountVectorizer()
# TODO
X_test_C = count_vector.transform(X_test)

print( X_train_C.shape)
print( X_test_C.shape)


(4179, 7456)
(1393, 7456)


 
### Étape 4 : Implémentation du théorème de Bayes à partir de zéro ###

Maintenant que nous avons notre ensemble de données dans le format dont nous avons besoin, nous pouvons passer à la prochaine partie de notre mission qui est l'algorithme que nous utiliserons pour faire nos prédictions pour classer un message comme spam ou non spam. Rappelez-vous qu'au début de la mission nous avons brièvement discuté du théorème de Bayes mais maintenant nous allons entrer un peu plus dans les détails. En termes simples, le théorème de Bayes calcule la probabilité qu'un événement se produise, sur la base de certaines autres probabilités liées à l'événement en question. Il est composé d'un a priori (les probabilités que nous connaissons ou qui nous sont données) et d'un a posteriori (les probabilités que nous cherchons à calculer à l'aide des a priori).

Implémentons le théorème de Bayes à partir de zéro en utilisant un exemple simple. Disons que nous essayons de trouver les chances qu'une personne souffre de diabète, étant donné qu'elle a été testée et qu'elle a obtenu un résultat positif.
Dans le domaine médical, ces probabilités jouent un rôle très important car elles traitent généralement de situations de vie ou de mort.

Nous supposons ce qui suit :

`P(D)` est la probabilité qu'une personne souffre de diabète. Sa valeur est « 0,01 » ou en d'autres termes, 1% de la population générale souffre de diabète.

`P(Pos)` est la probabilité d'obtenir un résultat de test positif.

`P(Neg)` est la probabilité d'obtenir un résultat de test négatif.

`P(Pos|D)` est la probabilité d'obtenir un résultat positif à un test effectué pour détecter le diabète, étant donné que vous êtes diabétique. Cela a une valeur « 0,9 ». En d'autres termes, le test est correct 90 % du temps. C'est ce qu'on appelle aussi la sensibilité ou le taux de vrais positifs.

`P(Neg|~D)` est la probabilité d'obtenir un résultat négatif à un test effectué pour détecter le diabète, étant donné que vous n'êtes pas diabétique. Cela a également une valeur de « 0,9 » et est donc correct, 90 % du temps. C'est ce qu'on appelle aussi la spécificité ou le taux de vrai négatif.

La formule de Bayes est la suivante :

<img src="images/bayes_formula.png" height="242" width="242">

* `P(A)` est la probabilité a priori que A se produise indépendamment. Dans notre exemple, il s'agit de `P(D)`. Cette valeur nous est donnée.

* `P(B)` est la probabilité a priori que B se produise indépendamment. Dans notre exemple, il s'agit de `P(Pos)`.

* `P(A|B)` est la probabilité a postérieuri que A se produise étant donné B. Dans notre exemple, c'est `P(D|Pos)`. C'est-à-dire ** la probabilité qu'un individu souffre de diabète, étant donné que cet individu a obtenu un résultat de test positif. C'est la valeur que nous cherchons à calculer.**

* `P(B|A)` est la probabilité que B se produise, étant donné A. Dans notre exemple, il s'agit de `P(Pos|D)`. Cette valeur nous est donnée.

En mettant nos valeurs dans la formule du théorème de Bayes, nous obtenons :

`P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)`

La probabilité d'obtenir un résultat de test positif `P(Pos)` peut être calculée en utilisant la sensibilité et la spécificité comme suit :

`P(Pos) = [P(D) * Sensitivity] + [P(~D) * (1-Specificity))]`

'''
Instructions:
Calculer la probabilité d'obtenir un résultat de test positif, P(Pos)
'''

In [22]:
# P(D)
p_diabetes = 0.01

# P(~D)
p_no_diabetes = 0.99

# Sensitivity or P(Pos|D)
p_pos_diabetes = 0.9

# Specificity or P(Neg/~D)
p_neg_no_diabetes = 0.9

# P(Pos)
p_pos = (p_diabetes * p_pos_diabetes) + (p_no_diabetes * (1 - p_neg_no_diabetes))
print('La probabilité d\'obtenir un résultat de test positif P(Pos) est :{}',format(p_pos))

La probabilité d'obtenir un résultat de test positif P(Pos) est :{} 0.10799999999999998


** En utilisant toutes ces informations, nous pouvons calculer nos postérieurs comme suit : **
    
La probabilité qu'un individu souffre de diabète, étant donné que cet individu obtient un résultat de test positif :

`P(D/Pos) = (P(D) * Sensitivity)) / P(Pos)`

La probabilité qu'une personne ne soit pas diabétique, étant donné que cette personne a obtenu un résultat de test positif :

`P(~D/Pos) = (P(~D) * (1-Specificity)) / P(Pos)`

La somme de nos postérieurs sera toujours égale à « 1 ».

'''
Instructions:
Calculez la probabilité qu'un individu souffre de diabète, étant donné que cet individu a obtenu un résultat de test positif.
En d'autres termes, calculez P(D|Pos).

La formule est : P(D|Pos) = (P(D) * P(Pos|D) / P(Pos)
'''

In [23]:
# P(D|Pos)
p_diabetes_pos = (p_diabetes *  p_pos_diabetes)/p_pos
print('La probabilité qu\'un individu souffre de diabète, étant donné que cet individu a obtenu un résultat de test positif est :\
',format(p_diabetes_pos)) 

La probabilité qu'un individu souffre de diabète, étant donné que cet individu a obtenu un résultat de test positif est : 0.08333333333333336



Instructions:

Calculez la probabilité qu'une personne ne souffre pas de diabète, étant donné que cette personne a obtenu un résultat de test positif.
En d'autres termes, calculez P(~D|Pos).

La formule est: P(~D|Pos) = (P(~D) * P(Pos|~D) / P(Pos)

Notez que P(Pos/~D) peut être calculé comme 1 - P(Neg/~D).

Par conséquent:
P(Pos/~D) = p_pos_no_diabetes = 1 - 0,9 = 0,1


In [24]:
# P(Pos/~D)
p_pos_no_diabetes = 0.1

# P(~D|Pos)
p_no_diabetes_pos =(p_no_diabetes * p_pos_no_diabetes) / p_pos
print ('La probabilité qu\'un individu ne souffre pas de diabète, étant donné que cet individu a obtenu un résultat de test positif est :'\
,p_no_diabetes_pos)

La probabilité qu'un individu ne souffre pas de diabète, étant donné que cet individu a obtenu un résultat de test positif est : 0.9166666666666669


Vous avez implémenté le théorème de Bayes à partir de zéro. Votre analyse montre que même si vous obtenez un résultat de test positif, il n'y a que 8,3 % de chances que vous soyez réellement diabétique et 91,67 % de chances que vous n'ayez pas de diabète. Cela suppose bien sûr que seulement 1% de la population totale souffre de diabète, ce qui n'est bien sûr qu'une hypothèse.

** Que signifie le terme 'Naive' dans 'Naive Bayes' ? **

Le terme « Naïf » dans Naive Bayes vient du fait que l'algorithme considère les caractéristiques qu'il utilise pour rendre les prédictions indépendantes les unes des autres, ce qui n'est pas toujours le cas. Ainsi, dans notre exemple sur le diabète, nous ne considérons qu'une seule caractéristique, à savoir le résultat du test. Supposons que nous ayons ajouté une autre fonctionnalité, « exercice ». Disons que cette caractéristique a une valeur binaire de « 0 » et « 1 », où la première signifie que l'individu s'exerce moins ou égal à 2 jours par semaine et la dernière signifie que l'individu exerce plus de ou égal à 3 jours par semaine la semaine. Si nous devions utiliser ces deux caractéristiques, à savoir le résultat du test et la valeur de la caractéristique « exercice », pour calculer nos probabilités finales, le théorème de Bayes échouerait. Naive Bayes est une extension du théorème de Bayes qui suppose que toutes les caractéristiques sont indépendantes les unes des autres.

### Étape 5 : Implémentation de Naive Bayes à l'aide de scikit-learn ###

Heureusement, sklearn dispose de plusieurs implémentations Naive Bayes que nous pouvons utiliser et nous n'avons donc pas à refaire le calcul à partir de zéro. Nous utiliserons la méthode sklearns `sklearn.naive_bayes` pour faire des prédictions sur notre ensemble de données.

Plus précisément, nous utiliserons l'implémentation `multinomial Naive Bayes`. Ce classificateur particulier convient à la classification avec des caractéristiques discrètes (comme dans notre cas, le nombre de mots pour la classification de texte). Il prend en entrée le nombre de mots entiers. D'autre part, Gaussian Naive Bayes est mieux adapté aux données continues car il suppose que les données d'entrée ont une distribution gaussienne (normale).


Instructions:

Nous avons chargé les données d'entraînement dans la variable "training_data" et les données de test dans la variable "testing_data".

Importez le classificateur MultinomialNB et ajustez les données d'apprentissage dans le classificateur à l'aide de fit(). Nommez votre classificateur 'naive_bayes'. Vous entraînerez le classificateur en utilisant 'training_data' et y_train' de notre division précédente.


In [25]:
from sklearn.naive_bayes import MultinomialNB
count_vectorizer = CountVectorizer()
training_data = count_vectorizer.fit_transform(X_train)
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

MultinomialNB()


Instructions:
Maintenant que notre algorithme a été entraîné à l'aide de l'ensemble de données d'entraînement, nous pouvons maintenant faire des prédictions sur les données de test
stocké dans 'testing_data' à l'aide de predict(). Enregistrez vos prédictions dans la variable « predictions ».


In [26]:
predictions = naive_bayes.predict(X_test_C)
predictions

array(['ham', 'ham', 'ham', ..., 'ham', 'spam', 'ham'], dtype='<U4')

Maintenant que les prédictions ont été faites sur notre ensemble de test, nous devons vérifier l'exactitude de nos prédictions.

### Étape 6 : Évaluer notre modèle ###

Maintenant que nous avons fait des prédictions sur notre ensemble de tests, notre prochain objectif est d'évaluer les performances de notre modèle. Il existe différents mécanismes pour le faire, mais commençons par les récapituler rapidement.

** `Accuracy` ** mesure la fréquence à laquelle le classificateur fait la prédiction correcte. C'est le rapport entre le nombre de prédictions correctes et le nombre total de prédictions (le nombre de points de données de test).

** `Precision` ** nous indique quelle proportion de messages que nous avons classés comme spam, étaient en fait du spam.
C'est un ratio de vrais positifs (mots classés comme spam, et qui sont en fait du spam) à tous les positifs (tous les mots classés comme spam, qu'il s'agisse ou non de la classification correcte), en d'autres termes c'est le ratio de

`[Vrais Positifs/(Vrais Positifs + Faux Positifs)]`

** Recall (sensibilité)** nous indique quelle proportion de messages qui étaient en fait du spam ont été classés par nous comme spam.
C'est un ratio de vrais positifs (mots classés comme spam, et qui sont en fait du spam) à tous les mots qui étaient réellement du spam, en d'autres termes c'est le ratio de

`[Vrais Positifs/(Vrais Positifs + Faux Négatifs)]`

Pour les problèmes de classification qui sont faussés dans leurs distributions de classification comme dans notre cas, par exemple si nous avions 100 messages texte et que seulement 2 étaient du spam et les 98 autres ne l'étaient pas, la précision en soi n'est pas une très bonne mesure. Nous pourrions classer 90 messages comme non spam (y compris les 2 qui étaient du spam mais nous les classons comme non spam, donc ils seraient de faux négatifs) et 10 comme spam (tous les 10 faux positifs) et obtenir toujours un score de précision raisonnablement bon. Pour de tels cas, la précision et le rappel sont très utiles. Ces deux mesures peuvent être combinées pour obtenir le score F1, qui est la moyenne pondérée des scores de précision et de rappel. Ce score peut aller de 0 à 1, 1 étant le meilleur score F1 possible.

Nous utiliserons les 4 métriques pour nous assurer que notre modèle fonctionne bien. Pour les 4 métriques dont les valeurs peuvent aller de 0 à 1, avoir un score aussi proche que possible de 1 est un bon indicateur de la performance de notre modèle.

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions, pos_label='spam')))
print('Recall score: ', format(recall_score(y_test, predictions, pos_label='spam')))
print('F1 score: ', format(f1_score(y_test, predictions, pos_label='spam')))

Accuracy score:  0.9885139985642498
Precision score:  0.9720670391061452
Recall score:  0.9405405405405406
F1 score:  0.9560439560439561


### Étape 7: Conclusion ###

L'un des principaux avantages de Naive Bayes par rapport aux autres algorithmes de classification est sa capacité à gérer un très grand nombre de fonctionnalités. Dans notre cas, chaque mot est traité comme une caractéristique et il existe des milliers de mots différents. En outre, il fonctionne bien même en présence de fonctionnalités non pertinentes et n'est relativement pas affecté par celles-ci. L'autre avantage majeur qu'il présente est sa relative simplicité. Naive Bayes fonctionne bien dès la sortie de la boîte et le réglage de ses paramètres est rarement nécessaire, sauf généralement dans les cas où la distribution des données est connue.
Il dépasse rarement les données. Un autre avantage important est que ses temps d'entraînement et de prédiction du modèle sont très rapides pour la quantité de données qu'il peut gérer. 